# Matmul results analysis

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def plt_clean_legend():
    original = plt.gca().get_legend_handles_labels()
    result = ([], [])
    for artist, label in zip(*original):
        try:
            if label == result[1][-1]:
                continue
        except:
            pass

        result[0].append(artist)
        result[1].append(label)
    plt.legend(result[0], result[1], loc="lower left")


In [ ]:
df = pd.read_csv("./matmul.csv") #, index_col=[0,1,2])
df

# Article image generation

In [ ]:
sns.set(style="whitegrid")

# First and last should be consistent, and reverse for consistency
palette = sns.color_palette(n_colors=5)
palette = list(reversed(palette))

ax = sns.barplot(x='mode', y='matmul',
                 data=df[(df.blocksize == 4096) & (df.matrixsize==7)],
                 capsize=0.1,
                 order=reversed(['mm', 'ad-dcnvm-resultnvm', 'ad-dcnvm-wram', 'ad-dcnvm-resultram', 'ad-ram']),
                 palette=palette)

ax.set_xticklabels(reversed(['mm', 'ad-resultnvm', 'ad-wram', 'ad-resultram', 'ad-ram']))

plt.ylabel("execution time (seconds)")
plt.xlabel(None)
plt.xticks(rotation=25, horizontalalignment='right')
plt.savefig("matmul-small.pdf", bbox_inches='tight')
plt.show()

ax = sns.barplot(x='mode', y='matmul',
                 data=df[(df.blocksize == 4096) & (df.matrixsize == 12)],
                 capsize=0.1,
                 order=reversed(['mm', 'ad-dcnvm-resultnvm', 'ad-dcnvm-wram', 'ad-dcnvm-resultram']),
                 palette=palette[1:]
                 )

ax.set_xticklabels(reversed(['mm', 'ad-resultnvm', 'ad-wram', 'ad-resultram']))

plt.ylabel("execution time (seconds)")
plt.xlabel(None)
plt.xticks(rotation=25, horizontalalignment='right')
plt.savefig("matmul-medium.pdf", bbox_inches='tight')
plt.show()

ax = sns.barplot(x='mode', y='matmul',
                 data=df[(df.blocksize == 4096) & (df.matrixsize == 14)],
                 capsize=0.1,
                 order=reversed(['mm', 'ad-dcnvm-resultnvm', 'ad-dcnvm-wram']),
                 palette=palette[2:])

ax.set_xticklabels(reversed(['mm', 'ad-resultnvm', 'ad-wram']))

plt.ylabel("execution time (seconds)")
plt.xlabel(None)
plt.xticks(rotation=25, horizontalalignment='right')
plt.savefig("matmul-big.pdf", bbox_inches='tight')
plt.show()


In [ ]:
df[(df.blocksize == 4096) & (df.matrixsize==7)].groupby("mode").mean()

In [ ]:
df[(df.blocksize == 4096) & (df.matrixsize == 12)].groupby("mode").mean()

In [ ]:
df[(df.blocksize == 4096) & (df.matrixsize == 14)].groupby("mode").mean()